# Preload libraries and functions
First of all, let's import libraries that will be used in this example

In [55]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as prep
import time

import matplotlib.pyplot as plt
# import tensorflow as tf
# from tensorflow import keras
#matplot inline


# np.set_printoptions(precision=8)

Define functions and class

In [56]:
class MethodException(Exception):
    def __init__(self, msg, value):
        self.msg=msg
        self.value=value
        
# def PolynomialFeature
def polyFeature(X, degree=2):
    poly = prep.PolynomialFeatures(degree,include_bias = False)
    polyArray = poly.fit_transform(X)
    return polyArray

def normalizeFeature(X, method):
    try:
        if X.ndim == 1:  # Reshape the 1d array
            X=np.reshape(X,(-1,1))
        X_norm=np.ones((X.shape[0],X.shape[1]+1), dtype=np.float64)
        if method == 'std':
            X_norm[:,1:]=(X-X.mean(0))/X.std(0)     
        elif method == 'range':
            X_norm[:,1:]=(X-X.min(0))/(X.max(0)-X.min(0))
        else:
            raise MethodException('method should be either \'std\' or \'range\'(case sensitive)', method)
    except MethodException as ex:
        print(f'The error is: {ex.msg}, here the input method is \'{ex.value}\'')
    else:
        return X_norm

# cost (error) function
def computeCost(X, y, theta):
    inner = np.power(((X @ theta.T) - y), 2)
    return np.sum(inner) / (2 * len(X))

# normal equation
def normalEq(X,y):
    theta=np.linalg.pinv(X.T@X)@X.T@y
    return theta

# normal equation with regulization
def normalRegEq(X,y,beta):
    L=np.eye(X.shape[1])
    L[0,0]=0
    theta=np.linalg.inv(X.T@X+beta*L)@X.T@y
    return theta
    
# def linearRegCostFunction(X, y, theta, reg):


# Data Import and Initialization

In [57]:
df = pd.read_excel('obs_data_w.xlsx', sheet_name=0) #sheet_name='Sheet1')
print(df)

         V    T  uncertainty             J
0     0.00  100     0.001000  0.000000e+00
1     0.05  100     0.001000  6.417136e-90
2     0.10  100     0.001000  1.081096e-88
3     0.15  100     0.001000  1.719633e-87
4     0.20  100     0.001000  2.725747e-86
5     0.25  100     0.001000  4.319561e-85
6     0.30  100     0.001000  6.845225e-84
7     0.35  100     0.001000  1.084764e-82
8     0.40  100     0.001000  1.719028e-81
9     0.45  100     0.001000  2.724148e-80
10    0.50  100     0.001000  4.316963e-79
11    0.55  100     0.001000  6.841100e-78
12    0.60  100     0.001000  1.084111e-76
13    0.65  100     0.001000  1.717992e-75
14    0.70  100     0.001000  2.722506e-74
15    0.75  100     0.001000  4.314361e-73
16    0.80  100     0.001000  6.836977e-72
17    0.85  100     0.001000  1.083457e-70
18    0.90  100     0.001000  1.716957e-69
19    0.95  100     0.001000  2.720865e-68
20    1.00  100     0.001000  4.311761e-67
21    1.05  100     0.001000  6.832857e-66
22    1.10 

## Train, cross validation and test data

### Random Method 1: Sample function of df

In [58]:
tcv=df.sample(frac=0.8) # random_state=200
test=df.drop(tcv.index)
test=test.sample(frac=1) # shuffle the rows

print(test.shape)

(1579, 4)


### Random Method 2: random array

In [ ]:
# idx=np.arange(0,len(df))
# np.random.shuffle(idx)
# temp_split=math.floor(0.8*len(df))
# #idx[0:20]
# train=df.iloc[idx[0:temp_split],:]
# test=df.iloc[idx[temp_split:],:] # Output all the elements after temp_split
# print(train.shape)
# print(test.shape)

## Split the data into data and results

In [59]:
test_data=test.loc[:,['V','T']]
test_unc=test.uncertainty.values
test_J=test.J.values

# print(test_data)
# J=J.tolist()

## Add the polynominal features

In [60]:
degree = 2
print(f'The polynominal degree is {degree}.\n')


# test data
test_data_poly = polyFeature(test_data, degree)
print('test data = \n', test_data_poly, '\n')


The polynominal degree is 2.

test data = 
 [[1.0000e-01 6.7000e+02 1.0000e-02 6.7000e+01 4.4890e+05]
 [1.4500e+00 1.7900e+02 2.1025e+00 2.5955e+02 3.2041e+04]
 [4.0000e-01 1.5400e+02 1.6000e-01 6.1600e+01 2.3716e+04]
 ...
 [1.3500e+00 1.1300e+02 1.8225e+00 1.5255e+02 1.2769e+04]
 [2.0000e+00 1.2900e+02 4.0000e+00 2.5800e+02 1.6641e+04]
 [1.1500e+00 1.8400e+02 1.3225e+00 2.1160e+02 3.3856e+04]] 



## Data Normalization (Two methods, range or stand deviation)  

(__We should try both__)

In [61]:
# Normalization method
method='std'

# test data
test_data_norm=normalizeFeature(test_data_poly, method)
print('normalized test_data = \n', test_data_norm, '\n')

# test the normalize method is right
# t=test_data_poly[:,1]
# print(t.ndim)
# t_norm=normalizeFeature(t,'std')
# print(t_norm)

normalized test_data = 
 [[ 1.         -1.518603    4.1398945  -1.04170296 -1.12970081  4.71219033]
 [ 1.         -0.24218248 -0.01634536 -0.49941088 -0.10177125 -0.17164876]
 [ 1.         -1.23495399 -0.22796653 -1.00282897 -1.15852875 -0.26918284]
 ...
 [ 1.         -0.33673215 -0.57502526 -0.57197565 -0.67299152 -0.39743577]
 [ 1.          0.2778407  -0.43958771 -0.00765498 -0.11004594 -0.35207217]
 [ 1.         -0.52583148  0.02597888 -0.7015556  -0.35775267 -0.15038457]] 



# Model Training

## Method 1: Normal Equation

### Training without regulation

In [ ]:
theta=normalEq(train_data_norm, train_J) 
print(f'theta = {theta}')
error_train=computeCost(train_data_norm, train_J, theta)
error_test=computeCost(test_data_norm, test_J, theta)

print('The training error is ', error_train)
print('The test error is ', error_test)

# OTHER PRINT EXPRESSION WITH FORMAT
# print('The training error is %.10f'%error)
# print('error = {:.10f}'.format(error))
# print(f'error = {error}')
# print('error= ', error)

### Training with regulation

In [64]:
featureSize=test_data_norm.shape[1]
beta_range=np.geomspace(0.001,15, num = 50-1, endpoint = True, dtype=np.float64)
beta_range=np.insert(beta_range,0,0) # insert a zero element in the first index
theta_reg=np.zeros((beta_range.size, featureSize))
error_train_reg=np.zeros(beta_range.size)
error_cv_reg=np.zeros(beta_range.size)

def train_model_loop(beta_num, epoch):
    
    # theta_reg_multismpl=np.zeros((epoch, featureSize))
    error_train_reg_multismpl=np.zeros(epoch)
    error_cv_reg_multismpl=np.zeros(epoch)
    beta_array=np.zeros(beta_num)
    for i in range(0,beta_num):
        for index, beta in enumerate(beta_range):
            for j in range(0,epoch,1):
                
                print(f'Loop {i}-{index}-{j} is running...\n')

                # Split into train set and cv set
                train=tcv.sample(frac=0.75, random_state = int((157*j+71*i)/3))
                cv=tcv.drop(train.index)

                train_data=train.loc[:,['V','T']]
                train_unc=train.uncertainty.values
                train_J=train.J.values

                cv_data=cv.loc[:,['V','T']]
                cv_unc=cv.uncertainty.values
                cv_J=cv.J.values

                # add polynominal features
                train_data_poly = polyFeature(train_data, degree)
                cv_data_poly = polyFeature(cv_data, degree)

                # train data normalization
                train_data_norm=normalizeFeature(train_data_poly, method)

                # cross valiation data normalization
                cv_data_norm=normalizeFeature(cv_data_poly, method)

                theta_reg_multismpl=normalRegEq(train_data_norm, train_J, beta)
                error_train_reg_multismpl[j]=computeCost(train_data_norm, train_J, theta_reg_multismpl)
                error_cv_reg_multismpl[j]=computeCost(cv_data_norm, cv_J, theta_reg_multismpl)
                
                print(f'Loop {i}-{index}-{j} ended.\n')

            error_train_reg[index] = error_train_reg_multismpl.sum(0)/epoch
            error_cv_reg[index] = error_cv_reg_multismpl.sum(0)/epoch
        idx=error_cv_reg.argmin()
        beta_array[i] = beta_range[idx]
    return beta_array
        



In [ ]:
t_start=time.time()
beta_array=train_model_loop(50, 100)
t_end=time.time()
print(f'The time cost is {t_end-t_start}s')

# plot error vs. beta
plt.figure()
plt.semilogx(beta_range, error_train_reg, label = 'Train error')
plt.semilogx(beta_range, error_cv_reg, label = 'Val error')
plt.xlabel('regulization coefficient (beta)')
plt.ylabel('error')
plt.legend()

# plot error vs. beta
plt.figure()
plt.plot(beta_range, error_train_reg, label = 'Train error')
plt.plot(beta_range, error_cv_reg, label = 'Val error')
plt.xlabel('regulization coefficient (beta)')
plt.ylabel('error')
plt.legend()

# print(error_cv_reg_multismpl.shape)
print(beta_array)
idx=error_cv_reg.argmin()
beta_best = beta_range[idx]
theta_best = theta_reg[idx]
error_test_reg=computeCost(test_data_norm, test_J, theta_best)

# print('The best value of THETA is ', theta_best)
print('The best value of BETA is ', beta_best)
# print('The test error with best THETA and BETA is ', error_test_reg)

# Prediction

In [67]:
beta_array.mean()

0.9674956991442808

In [68]:
print(beta_array)

[0.04867541 1.60652778 1.08932452 0.37423736 0.67026043 1.08932452
 0.73862894 2.36929533 0.18961293 2.1499901  1.08932452 1.45782535
 0.98849514 0.         0.41241066 1.95098405 0.         0.
 0.         1.45782535 0.81397124 1.77039828 0.         1.32288703
 0.5519225  0.         0.50083579 0.89699867 1.32288703 2.1499901
 0.98849514 2.1499901  0.27963976 1.77039828 0.45447774 1.20043879
 0.         1.08932452 2.1499901  1.95098405 1.60652778 0.
 0.         2.61097035 1.32288703 0.1720621  0.30816383 0.50083579
 1.60652778 1.20043879]


In [ ]:
print(t.array)

In [ ]:
t.polyFeature(2)
t.array.shape

In [14]:
x=1

2
1


UnboundLocalError: local variable 'x' referenced before assignment